In [ ]:
import os
import shutil
import time
import pandas as pd
from PIL import Image
from pathlib import Path
from IPython.display import clear_output

In [ ]:
res_pic_folder = "./result_pic"
ng_pic_folder = "./result_NG"
rows, cols = 3,3

In [ ]:
# Init
os.makedirs(res_pic_folder, exist_ok=True)
os.makedirs(ng_pic_folder, exist_ok=True)
todo_list = [(stage,f) for stage,_,files in os.walk(".") for f in files if (".png" in f) and "result" not in stage]
num_pics = len(todo_list)
batch_size = rows*cols
num_batch = (num_pics+batch_size-1) // batch_size
df_result = pd.DataFrame(columns=["Stage", "Filename", "LotID", "NG", "Batch"])

print(f'Total {num_pics} png files, See {num_batch} Batch')
print(f'Todo List : {todo_list}')

In [ ]:
def lot_chunker(todo_list, batch_size):
    start, end = 0, batch_size
    while end < len(todo_list):
        yield todo_list[start:end]
        start, end = end, end+batch_size
    yield todo_list[start:len(todo_list)]
    
for batch_id, _lot_list in enumerate(lot_chunker(todo_list, batch_size)):
    clear_output(wait=True)
    stage_list, filename_list, lotid_list = [], [], []
    ng_list, batch = [False]*len(_lot_list), [batch_id]*len(_lot_list)
    dic_batch = {"Stage": stage_list, "Filename": filename_list, "LotID": lotid_list, "NG": ng_list, "Batch": batch}
    
    _imgs = []
    for _stage, _file in _lot_list:
        path = Path(_stage)/_file
        _imgs.append(Image.open(path))
        
        stage_list.append(_stage)
        filename_list.append(_file)
        _lot = _file.split("_",maxsplit=1)[1].split(".png")[0]
        lotid_list.append(_lot)
    
    print(f'Lot: {lotid_list}')
    width, height = _imgs[0].size
    new_img = Image.new('RGB', ((width+50)*cols-50, (height+50)*rows-50))
    pos_x, pos_y = 0, 0
    for idx, img in enumerate(_imgs):
        pos_x = (idx%cols) * (width+50)
        pos_y = (idx//cols) * (height+50)
        new_img.paste(img, (pos_x, pos_y))
    new_img_path = Path(res_pic_folder)/f'Batch_{batch_id:02d}.png'
    new_img.save(new_img_path)
    
    display(new_img)
    time.sleep(0.2*batch_size)
    
    ng_idx = input()
    if ng_idx != "":
        for _idx in ng_idx.split(","):
            idx = int(_idx)-1
            ng_list[idx] = True
            
            _stage, _file = stage_list[idx], filename_list[idx]
            src_path = Path(_stage)/_file
            dst_path = Path(ng_pic_folder) / _stage / _file
            os.makedirs(os.path.dirname(dst_path), exist_ok=True)
            shutil.copy(src_path, dst_path)
    
    df_batch = pd.DataFrame.from_dict(dic_batch)
    df_result = pd.concat([df_result, df_batch], axis=0).reset_index(drop=True)
    df_result.to_csv("./res.csv", index=False)
    
clear_output(wait=True)
df_result